Aula 1
Pré-requisitos Conta ativa no GEE, como criar: https://earthengine.google.com/new_signup/

### Carregando as bibliotecas necessárias no python

O curso será todo baseado na biblioteca geemap, que contém diversas ferramentas para utilização do gee no python.
Maiores informações:
https://geemap.org/ 

In [ ]:
%%capture 
# O comando capture é utilizado para que os outputs da célula não sejam exibidos, se quiser exibir, é só comentar a linha
!pip install geemap # pip é o comando de instalação no python, quando vc coloca a ! na frente do comando, vc está executando como se fosse no prompt de comando

In [ ]:
# import geemap #aqui vc está importando a biblioteca
import ee

In [ ]:
import geemap

In [ ]:
Map = geemap.Map() #esse comando vai exigir uma autenticação


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=z4YQkdGhcr0r4O1aTQZeu12uQ6LbaccFoXu_2RtgWM0&tc=xpPjZDU6rk4JJyuUVONbhWoseQUgkJG10wGMty080RA&cc=96LmHpHFlvK0tcol6wToN-3iM_eyQAz-iBij2orm6wY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgCdmt-gIL0ORaTf1rhTuF7AlAHm2jpduySHc6EG6VaFJs2zNqpKGs

Successfully saved authorization token.


### Área de estudo 
*Abreviações comuns: AOI ou ROI*

Seleção de AE de estudo	

Exemplos de criação de um polígono para a AE de estudo

**Opção 1**
*   Desenhar a AE no mapa
*   Verificar as propriedades do polígono
*   Mudar o nome da variável para 'AE'

In [ ]:
Map
#explorar funcionalidades interativas e desenhar AOI

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
roi = ee.FeatureCollection(Map.draw_features)

In [ ]:
roi.getInfo()

In [ ]:
# Exportar o que vc desenhou ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toAsset(**{
    'collection': roi,
    'description':'exportToTableAssetExample',
    'assetId': 'projects/ee-atoro/assets/aula1' #aqui eu suponho que dê pra colocar tds em uma collection
})
task.start()

In [ ]:
task.status() #acompanhar o status

In [ ]:
roi = ee.FeatureCollection('projects/ee-atoro/assets/aula1')

In [ ]:
# Se quiser, pode centralizar na região de acordo com as coordenadas
Map.setCenter(-55.8723, -11.6488) #pode centralizar assim ou pela ferramenta interativa no canto superior esquerdo

In [ ]:
a = ee.Geometry(roi.geometry())

In [ ]:
print('Polygon area: ', a.area().divide(1000 * 1000).getInfo()) # in square kilometers


### 2. Coleção de imagem

In [ ]:

AE = roi.geometry()

In [ ]:
L5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(AE).filterDate('2010-11-15', '2011-03-01')
img_estudo = ee.Image(L5.median()).clip(AE)
visParams = ({'bands':['B3',  'B2',  'B1'], 'min': 150, 'max': 2000})


In [ ]:
# //Adicionar as cenas de acordo com a AE de estudo 
Map.addLayer(L5.first(), visParams, 'Imagem Landsat 5')

Map.addLayer(img_estudo, visParams, 'Imagem Landsat 5 - AE de estudo')

In [ ]:
Map

In [ ]:

# Visualizar a lista de imagens
ListadeImagens_L5 = L5.toList(L5.size())
print('ListadeImagens_L5:', ListadeImagens_L5.getInfo())

In [ ]:
#Para LANDSAT 8 - disponibilidade de Abril 2013 até o momento

In [ ]:
 #coleção do Landsat 8
 L8 = ee.ImageCollection ('LANDSAT/LC08/C01/T1_SR').filterBounds(AE).filterDate('2016-11-15', '2017-03-01')

In [ ]:
# //Adicionar as cenas de acordo com a AE de estudo 
Map = geemap.Map()

img_estudo = ee.Image(L8.median()).clip(AE)
visParams = ({'bands':['B4',  'B3',  'B2'], 'min': 150, 'max': 2000})
Map.addLayer(L8.first(), visParams, 'Imagem Landsat 8')

Map.addLayer(img_estudo, visParams, 'Imagem Landsat 8 - AE de estudo')
# Map

In [ ]:
Map

In [ ]:
#Recorte ae de estudo

In [ ]:
# // Criando função clip para recortar a AE de estudo
# //Para L5 substituir L8 por L8 
Map = geemap.Map()

def clipper(image):
  return image.clip(AE)

L8 = L8.map(clipper)
Map.addLayer(L8)
Map